# Ejercicio 1: Introducción a Recuperación de Información

## Objetivo de la práctica
- Entender el problema de **buscar información** en colecciones de texto.
- Comprender por qué se necesita un **índice invertido** en recuperación de información.
- Programar una primera solución manual y luego optimizarla con un índice.
- Evaluar la mejora en tiempos de búsqueda cuando usamos estructuras adecuadas.

## Parte 1: Búsqueda lineal en documentos

### Actividad
1. Se te proporcionará un conjunto de documentos de texto.
2. Escribe una función que:
   - Lea todos los documentos.
   - Busque una palabra ingresada por el usuario.
   - Muestre en qué documentos aparece la palabra.

In [16]:
ruta = '/content/docs/01_corpus_turismo.txt'
query = input("Ingresa la palabra a buscar: ")

documentos_encontrados = []

with open(ruta, 'r', encoding='utf-8') as f:
    lineas = f.readlines()
    for i, linea in enumerate(lineas):
        if query in linea:
            documentos_encontrados.append(f"Línea {i+1}")

if documentos_encontrados:
    print(f"La palabra '{query}' aparece en las siguientes líneas del documento:")
    for doc in documentos_encontrados:
        print(doc)
else:
    print(f"La palabra '{query}' no se encuentra en el documento.")


Ingresa la palabra a buscar: Quito
La palabra 'Quito' aparece en las siguientes líneas del documento:
Línea 3


## Parte 2: Construcción de un índice invertido

### Actividad
1. Escribe un programa que:
   - Recorra todos los documentos.
   - Construya un **índice invertido**, es decir, un diccionario donde:
     - Cada palabra clave apunta a una lista de documentos donde aparece.

2. Escribe una nueva función de búsqueda que:
   - Consulte directamente el índice para encontrar los documentos relevantes.
   - Sea mucho más rápida que la búsqueda lineal.

In [15]:
ruta = '/content/docs/01_corpus_turismo.txt'

def construir_indice_invertido(archivo):
    indice = {}
    with open(archivo, 'r', encoding='utf-8') as f:
        lineas = f.readlines()
        for i, linea in enumerate(lineas):
            palabras = linea.split()
            for palabra in set(palabras):
                if palabra not in indice:
                    indice[palabra] = []
                indice[palabra].append(f"linea {i+1}")
    return indice

def buscar_palabra(indice, palabra):
    palabra = palabra
    return indice.get(palabra, [])

indice_invertido = construir_indice_invertido(ruta)

query = input()
documentos = buscar_palabra(indice_invertido, query)

if documentos:
    for doc in documentos:
        print(doc)
else:
    print("No encontrado")


Quito
linea 3


## Parte 3: Evaluación de tiempos de búsqueda
### Actividad

1. Realiza la búsqueda de varias palabras usando:
      -  Corpus pequeño: 16 documentos (turismo en Ecuador).
      -  Corpus grande: 500 documentos (versión ampliada).
2. Mide el tiempo de ejecución:
      -  Para búsqueda lineal.
      -  Para búsqueda usando índice invertido.
      -  Grafica o presenta los resultados en una tabla comparativa.

### Ejemplo de palabras para buscar
- quito
- montañita
- feriado
- playas
- aventura
- galápagos

Corpus pequeño: Busqueda lineal

In [18]:
import time
import pandas as pd

ruta = '/content/docs/01_corpus_turismo.txt'

queries = ["quito", "montañita", "feriado", "playas", "aventura", "galápagos"]

resultados_lineal = []

for query in queries:
    documentos_encontrados = []

    inicio = time.time()

    with open(ruta, 'r', encoding='utf-8') as f:
        lineas = f.readlines()
        for i, linea in enumerate(lineas):
            if query in linea:
                documentos_encontrados.append(f"Línea {i+1}")

    fin = time.time()

    resultados_lineal.append({
        'Palabra': query,
        'Documentos encontrados': documentos_encontrados if documentos_encontrados else "No encontrado",
        'Tiempo de búsqueda (s)': round(fin - inicio, 6)
    })

df_lineal = pd.DataFrame(resultados_lineal)
df_lineal


,Palabra,Documentos encontrados,Tiempo de búsqueda (s)
0,quito,No encontrado,0.000101
1,montañita,No encontrado,0.000094
2,feriado,"[Línea 12, Línea 14, Línea 16]",0.000041
3,playas,"[Línea 1, Línea 5, Línea 10]",0.000032
4,aventura,[Línea 4],0.000030
5,galápagos,No encontrado,0.000027


Corpus pequeño: Indice invertido

In [19]:
import time
import pandas as pd

ruta = '/content/docs/01_corpus_turismo.txt'

def construir_indice_invertido(archivo):
    indice = {}
    with open(archivo, 'r', encoding='utf-8') as f:
        lineas = f.readlines()
        for i, linea in enumerate(lineas):
            palabras = linea.split()
            for palabra in set(palabras):
                if palabra not in indice:
                    indice[palabra] = []
                indice[palabra].append(f"Línea {i+1}")
    return indice

def buscar_palabra(indice, palabra):
    return indice.get(palabra, [])

inicio_indice = time.time()
indice_invertido = construir_indice_invertido(ruta)
fin_indice = time.time()

print(f"\nTiempo para construir el índice: {fin_indice - inicio_indice:.6f} segundos")

queries = ["quito", "montañita", "feriado", "playas", "aventura", "galápagos"]

resultados_invertido = []

for query in queries:
    inicio = time.time()

    documentos = buscar_palabra(indice_invertido, query)

    fin = time.time()

    resultados_invertido.append({
        'Palabra': query,
        'Documentos encontrados': documentos if documentos else "No encontrado",
        'Tiempo de búsqueda (s)': round(fin - inicio, 6)
    })

df_invertido = pd.DataFrame(resultados_invertido)
df_invertido



Tiempo para construir el índice: 0.001614 segundos


,Palabra,Documentos encontrados,Tiempo de búsqueda (s)
0,quito,No encontrado,0.000003
1,montañita,No encontrado,0.000001
2,feriado,[Línea 14],0.000001
3,playas,"[Línea 5, Línea 10]",0.000001
4,aventura,[Línea 4],0.000001
5,galápagos,No encontrado,0.000001


Tabla comparativa

| Palabra    | Tiempo Búsqueda Lineal (s) | Tiempo Búsqueda Índice Invertido (s) |
|------------|----------------------------|-------------------------------------|
| quito      | 0.000101                   | 0.000003                            |
| montañita  | 0.000094                   | 0.000001                            |
| feriado    | 0.000041                   | 0.000001                            |
| playas     | 0.000032                   | 0.000001                            |
| aventura   | 0.000030                   | 0.000001                            |
| galápagos  | 0.000027                   | 0.000001                            |


Corpus grande: Busqueda lineal

In [20]:
import time
import pandas as pd

ruta = '/content/docs/01_corpus_turismo_500.txt'

queries = ["quito", "montañita", "feriado", "playas", "aventura", "galápagos"]

resultados_lineal = []

for query in queries:
    documentos_encontrados = []

    inicio = time.time()

    with open(ruta, 'r', encoding='utf-8') as f:
        lineas = f.readlines()
        for i, linea in enumerate(lineas):
            if query in linea:
                documentos_encontrados.append(f"Línea {i+1}")

    fin = time.time()

    resultados_lineal.append({
        'Palabra': query,
        'Documentos encontrados': documentos_encontrados if documentos_encontrados else "No encontrado",
        'Tiempo de búsqueda (s)': round(fin - inicio, 6)
    })

df_lineal = pd.DataFrame(resultados_lineal)
df_lineal

,Palabra,Documentos encontrados,Tiempo de búsqueda (s)
0,quito,No encontrado,0.001843
1,montañita,No encontrado,0.000398
2,feriado,"[Línea 9, Línea 10, Línea 12, Línea 16, Línea ...",0.000305
3,playas,"[Línea 20, Línea 28, Línea 36, Línea 51, Línea...",0.000254
4,aventura,"[Línea 38, Línea 54, Línea 62, Línea 77, Línea...",0.000228
5,galápagos,No encontrado,0.000254


Corpus grande: Indice invertido

In [21]:
import time
import pandas as pd

ruta = '/content/docs/01_corpus_turismo_500.txt'

def construir_indice_invertido(archivo):
    indice = {}
    with open(archivo, 'r', encoding='utf-8') as f:
        lineas = f.readlines()
        for i, linea in enumerate(lineas):
            palabras = linea.split()
            for palabra in set(palabras):
                if palabra not in indice:
                    indice[palabra] = []
                indice[palabra].append(f"Línea {i+1}")
    return indice

def buscar_palabra(indice, palabra):
    return indice.get(palabra, [])

inicio_indice = time.time()
indice_invertido = construir_indice_invertido(ruta)
fin_indice = time.time()

print(f"\nTiempo para construir el índice: {fin_indice - inicio_indice:.6f} segundos")

queries = ["quito", "montañita", "feriado", "playas", "aventura", "galápagos"]

resultados_invertido = []

for query in queries:
    inicio = time.time()

    documentos = buscar_palabra(indice_invertido, query)

    fin = time.time()

    resultados_invertido.append({
        'Palabra': query,
        'Documentos encontrados': documentos if documentos else "No encontrado",
        'Tiempo de búsqueda (s)': round(fin - inicio, 6)
    })

df_invertido = pd.DataFrame(resultados_invertido)
df_invertido


Tiempo para construir el índice: 0.005074 segundos


,Palabra,Documentos encontrados,Tiempo de búsqueda (s)
0,quito,No encontrado,0.000002
1,montañita,No encontrado,0.000001
2,feriado,No encontrado,0.000000
3,playas,"[Línea 20, Línea 28, Línea 36, Línea 51, Línea...",0.000000
4,aventura,"[Línea 38, Línea 54, Línea 62, Línea 77, Línea...",0.000000
5,galápagos,No encontrado,0.000001


Tabla comparativa

| Palabra    | Tiempo Búsqueda Lineal (s) | Tiempo Búsqueda Índice Invertido (s) |
|------------|----------------------------|-------------------------------------|
| quito      | 0.001843                   | 0.000002                            |
| montañita  | 0.000398                   | 0.000001                            |
| feriado    | 0.000305                   | 0.000000                            |
| playas     | 0.000254                   | 0.000000                            |
| aventura   | 0.000228                   | 0.000000                            |
| galápagos  | 0.000254                   | 0.000001                            |


## Parte 4:
### Actividad
1. Modifica el índice para que ignore mayúsculas/minúsculas (por ejemplo, "Playa" y "playa" deben considerarse iguales).
2. Permite consultas de múltiples términos (ejemplo: buscar documentos que contengan "playa" y "turismo").
3. Calcula el _speedup_

Busqueda lineal

In [24]:
import time

ruta = '/content/docs/01_corpus_turismo.txt'

with open(ruta, 'r', encoding='utf-8') as f:
    lineas = f.readlines()

query = input("Ingresa las palabras a buscar (separadas por espacio): ").lower()
terminos = query.split()

documentos_encontrados = []
inicio = time.time()

for i, linea in enumerate(lineas):
    linea_lower = linea.lower()
    if all(termino in linea_lower for termino in terminos):
        documentos_encontrados.append(f"Línea {i+1}")

fin = time.time()
tiempo_busqueda_lineal = fin - inicio

if documentos_encontrados:
    print(f"Las palabras '{query}' aparecen en las siguientes líneas del documento:")
    for doc in documentos_encontrados:
        print(doc)
else:
    print(f"Las palabras '{query}' no se encuentran en el documento.")

print(f"Tiempo de búsqueda lineal: {tiempo_busqueda_lineal:.6f} segundos")


Ingresa las palabras a buscar (separadas por espacio): quito cuenta
Las palabras 'quito cuenta' aparecen en las siguientes líneas del documento:
Línea 3
Tiempo de búsqueda lineal: 0.000376 segundos


Indice invertido

In [25]:
import time

ruta = '/content/docs/01_corpus_turismo.txt'

def construir_indice_invertido(archivo):
    indice = {}
    with open(archivo, 'r', encoding='utf-8') as f:
        lineas = f.readlines()
        for i, linea in enumerate(lineas):
            palabras = linea.lower().split()
            for palabra in set(palabras):
                if palabra not in indice:
                    indice[palabra] = []
                indice[palabra].append(f"Línea {i+1}")
    return indice

inicio_construccion = time.time()
indice_invertido = construir_indice_invertido(ruta)
fin_construccion = time.time()
tiempo_construccion = fin_construccion - inicio_construccion

query = input("Ingresa las palabras a buscar (separadas por espacio): ").lower()
terminos = query.split()

inicio_busqueda = time.time()

listas = [set(indice_invertido.get(termino, [])) for termino in terminos]
lineas_comunes = set.intersection(*listas) if listas else set()

fin_busqueda = time.time()
tiempo_busqueda_indice = fin_busqueda - inicio_busqueda

if lineas_comunes:
    print(f"Las palabras '{query}' aparecen en las siguientes líneas del documento:")
    for doc in lineas_comunes:
        print(doc)
else:
    print(f"Las palabras '{query}' no se encuentran en el documento.")

print(f"Tiempo de construcción del índice: {tiempo_construccion:.6f} segundos")
print(f"Tiempo de búsqueda con índice invertido: {tiempo_busqueda_indice:.6f} segundos")


Ingresa las palabras a buscar (separadas por espacio): quito cuenta
Las palabras 'quito cuenta' aparecen en las siguientes líneas del documento:
Línea 3
Tiempo de construcción del índice: 0.001603 segundos
Tiempo de búsqueda con índice invertido: 0.000194 segundos


In [26]:
speedup = tiempo_busqueda_lineal / tiempo_busqueda_indice
print(f"Speedup de la búsqueda con índice invertido sobre búsqueda lineal: {speedup:.2f}x más rápido")


Speedup de la búsqueda con índice invertido sobre búsqueda lineal: 1.94x más rápido
